In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')
db = client.db_cdiscount

In [2]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [3]:
import numpy as np

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import keras
K.set_image_dim_ordering('th')

In [12]:
first = 0
num_batch = 10
last = 82
# last = 100000

In [13]:
epochs = 10
num_classes = db.cat_encode.count()

In [14]:
def simple_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(40, (3,3), input_shape = (180,180,3), activation='relu'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
model =simple_model(epochs=epochs, num_classes=num_classes)

In [18]:
counter = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
    lst_batch = []
    ## training Data
    for i in (db.train_example.find({})[a1:an]):
        dic = {}
    #     print('category_id: ', i['category_id'])
        dic['category_id'] = i['category_id']

    #     print('cat: ', db.cat_encode.find_one({ "cat" : (1000004079)}, {"cat" : 1.0, "_id" : 0})['cat'])
        dic['cat'] = db.cat_encode.find_one({ "cat" : (i['category_id'])}, {"cat" : 1.0, "_id" : 0})['cat']

    #     print('img: ', i['imgs'])
#         dic['imgs'] = i['imgs']
        picture = imread(io.BytesIO(i['imgs'][0]['picture']))
    ## divide image by 255.0
        picture = picture/255.0
        dic['picture'] = picture.reshape(180,180,3)

    #     print('encode: ', db.cat_encode.find_one({ "cat" : (1000004079)}, {"encode" : 1.0, "_id" : 0})['encode'])
        dic['encode'] = db.cat_encode.find_one({ "cat" : (i['category_id'])}, {"encode" : 1.0, "_id" : 0})['encode']

        lst_batch.append(dic)
             
        counter = counter +1
        
    
    X_batch = np.array([lst['picture'] for lst in lst_batch])
    Y_batch = np.array([lst['encode'] for lst in lst_batch])
    print(X_batch.shape)
    print(Y_batch.shape)
    model.fit(X_batch, Y_batch, batch_size=512, epochs=epochs, validation_split=0.2)
    #, validation_split=0.2
print(an)

(9, 180, 180, 3)
(9, 5270)
Train on 7 samples, validate on 2 samples
Epoch 1/10
7/7 [==============================] - 0s - loss: 8.5910 - acc: 0.0000e+00 - val_loss: 7.2227 - val_acc: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 0s - loss: 3.9557 - acc: 0.1429 - val_loss: 10.0576 - val_acc: 0.0000e+00
Epoch 3/10
7/7 [==============================] - 0s - loss: 3.6682 - acc: 0.2857 - val_loss: 8.7932 - val_acc: 0.0000e+00
Epoch 4/10
7/7 [==============================] - 0s - loss: 5.3156 - acc: 0.4286 - val_loss: 12.8152 - val_acc: 0.0000e+00
Epoch 5/10
7/7 [==============================] - 0s - loss: 7.4750 - acc: 0.4286 - val_loss: 9.8012 - val_acc: 0.0000e+00
Epoch 6/10
7/7 [==============================] - 0s - loss: 6.3546 - acc: 0.1429 - val_loss: 8.0603 - val_acc: 0.5000
Epoch 7/10
7/7 [==============================] - 0s - loss: 6.9676 - acc: 0.2857 - val_loss: 12.1952 - val_acc: 0.0000e+00
Epoch 8/10
7/7 [==============================] - 0s - loss: 6.2581

7/7 [==============================] - 0s - loss: 11.5129 - acc: 0.2857 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/10
7/7 [==============================] - 0s - loss: 11.5129 - acc: 0.2857 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/10
7/7 [==============================] - 0s - loss: 11.5129 - acc: 0.2857 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/10
7/7 [==============================] - 0s - loss: 11.5129 - acc: 0.2857 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 7/10
7/7 [==============================] - 0s - loss: 11.5129 - acc: 0.2857 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 8/10
7/7 [==============================] - 0s - loss: 11.5129 - acc: 0.2857 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 9/10
7/7 [==============================] - 0s - loss: 11.5129 - acc: 0.2857 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 10/10
7/7 [==============================] - 0s - loss: 11.5129 - acc: 0.2857 - val_loss: 1.1921e-07 - val_acc: 1.0000
(9, 180, 180, 3)
(

In [ ]:
# len(np.unique([cat['category_id'] for cat in lst_batch]))

In [ ]:
# num_classes=len(dummy_y[81])
# epochs = 5